# 简单App

在 [Learn Docker](https://github.com/tryk8s/learn-docker) 中，我们使用了 [Flask](http://flask.pocoo.org/) 的一款Demo程序 [Flaskr](https://github.com/pallets/flask/tree/master/examples/flaskr) 来演示如何将一个App打包成镜像并运行在Docker环境当中。

今天我们将使用同样的App作为案例，来演示在Kubernetes集群中运行一个App的流程。请注意比较与Docker案例在流程上的区别。

## 生成镜像

与Docker案例当中一样，我们首先生成镜像。我们会使用到`flaskr`与`nginx`两个镜像。

In [ ]:
!docker build -t registry:5000/flaskr flaskr/

In [ ]:
!docker build -t registry:5000/flaskr-nginx nginx/

## 将镜像推送到镜像仓库

推送的操作也非常简单，这里就不多做解释了。

In [ ]:
!docker push registry:5000/flaskr

In [ ]:
!docker push registry:5000/flaskr-nginx

## 创建Replication Controller

接下来让我们来试着操作Kubernetes集群。首先创建Replication Controller的定义文件。在这里我们已经提前创建好了

In [ ]:
!cat flaskr-rc.yaml

In [ ]:
!cat flaskr-nginx-rc.yaml

可以看到两个Replication Controller的定义是非常简单和类似的。主要的参数有3个，`selector`是Controller用来检查的查询条件，`template`是Controller启动的Pod的模板，`replicas`表示需要启动几个Pod。Controller会不断用查询条件进行搜索。如果当前Pod数低于`replicas`数，则使用`template`创建新的Pod，如果Pod数高于`replicas`数，则停掉多余的Pod。

接下来将这两个定义文件发送到Kubernetes集群。

In [ ]:
!kubectl create -f flaskr-rc.yaml

In [ ]:
!kubectl create -f flaskr-nginx-rc.yaml

## 创建Service

创建Service的流程与创建Replication Controller的流程类似。还是先来看看定义文件

In [ ]:
!cat flaskr-svc.yaml

In [ ]:
!cat flaskr-nginx-svc.yaml

Service定义中的主要参数有2个，`selector`是Service所对应的Pod的查询条件，`ports`是端口的定义。当有连接试图访问这个Service时，Service会将访问请求转发到`selector`所指定的Pod组当中，并在这组Pod当中做负载均衡。具体的Service端口以及Pod上的端口信息，在`ports`当中进行了定义。

需要指出的是Service定义当中的`name`字段将会被用在集群内部基于dns的服务发现系统当中。

将服务定义发往集群，完成创建

In [ ]:
!kubectl create -f flaskr-svc.yaml

In [ ]:
!kubectl create -f flaskr-nginx-svc.yaml

这时，我们可以使用浏览器访问 `http://$SERVER_IP:30080` 确认flaskr服务已经成功运行起来了。

## 负载均衡

到目前为止，我们在Docker环境和Kubernetes环境中部署flaskr系统的操作步骤还是比较接近的。接下来我们做一些Kubernetes集群当中所独有的操作。

由于我们将所有的flaskr Pod作为一组，用Service暴露给nginx。同时也将nginx Pod作为一组，用Service暴露给外部用户。我们可以用一个指令启动更多的副本来承载更多的流量。

In [ ]:
!kubectl scale rc flaskr --replicas=3

In [ ]:
!kubectl scale rc flaskr-nginx --replicas=2

在这里我们启动了2个nginx副本，3个flaskr副本。一来避免了单点故障，二来提高了性能。这些副本将由Kubernetes调度到集群当中的某一个节点运行。同时当节点出现故障时，Kubernetes将会把上面运行的程序迁移到其他节点上。而这一切都是自动完成，并且不会产生业务中断。

## 总结

在这个案例中，我们使用Kubernetes重新实现了flaskr系统的部署。利用Kubernetes实现了简单的高可用以及服务处理能力扩展。

## 资源释放

在完成这个案例后，我们将刚才创建的资源释放出来，供后面的案例使用

In [ ]:
!kubectl delete rc flaskr

In [ ]:
!kubectl delete rc flaskr-nginx

In [ ]:
!kubectl delete svc flaskr

In [ ]:
!kubectl delete svc flaskr-nginx